In [ ]:
!pip install -q -U google-generativeai

In [111]:

!pip install -q google-cloud-aiplatform==1.36.0 google-cloud-discoveryengine==0.11.2 langchain==0.0.327 p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.47 requires langchain<0.1,>=0.0.350, but you have langchain 0.0.327 which is incompatible.


In [ ]:
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai


In [1]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [1]:
import pathlib
import textwrap
import os
import google.generativeai as genai
import pathlib
import textwrap
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
load_dotenv()
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
model = genai.GenerativeModel('gemini-pro')

In [27]:
# %%time
# responce=model.generate_content("What is the meaning of life?")

In [28]:
# to_markdown(response.text)
# print(responce.text)

The meaning of life is a topic that has been contemplated by philosophers, theologians, and ordinary people for centuries. There is no one definitive answer that is universally accepted, as it is a highly personal and subjective matter. However, here are some common perspectives on the meaning of life:

1. **Purposeful Engagement**: Many people find meaning in life through purposeful engagement in activities that they find fulfilling and rewarding. This could involve pursuing a career or hobby that aligns with their interests and values, engaging in creative endeavors, or contributing to society through volunteer work or other forms of service.

2. **Relationships and Connections**: For many people, strong relationships and meaningful connections with others are central to finding meaning in life. This can include spending time with loved ones, building close friendships, and forming supportive and enriching relationships with colleagues, neighbors, and community members.

3. **Persona

RAG(Lanchain+chroma+GeminiAI)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [19]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate

# from langchain.schema.output_parser import StrOutputParser

In [6]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [9]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings)

#RAG template class for TRue Lense

In [10]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [11]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        retriever = vectordb.as_retriever()
        results = retriever.get_relevant_documents(query) 
        return results[0]  

    
    @instrument
    def generate_completion(self, query:str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        message = [{
            "role": "user",
            "parts": [
                f"We have provided context information below. \n"
                f"---------------------\n"
                f"{context_str}"
                f"\n---------------------\n"
                f"Given this information, please answer the question: {query}"
            ]
        }]
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(message)
        response_txt = response.text
        return response_txt
        

    @instrument
    def query(self, query:str ) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion
    
rag = RAG_from_scratch()


In [38]:
# # Usage
# rag_instance = RAG_from_scratch()
# my_query = "What is the news about Pando?"
# result = rag_instance.query(my_query)
# print(result)



In [39]:
#Evaluation using TrueLens


In [24]:
# !pip install trulens-eval

In [25]:
!pip install streamlit_javascript


Defaulting to user installation because normal site-packages is not writeable
  Using cached streamlit_javascript-0.1.5-py3-none-any.whl (518 kB)


In [26]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [27]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np
import streamlit as st


In [28]:
# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [29]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [30]:
with tru_rag as recording:
    rag.query("What is the news about Pando")

In [31]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
RAG v1,1.0,0.8,0.48125,2.307692,0.0


In [32]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.10:8502



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>